In [1]:
%matplotlib widget
import ipywidgets as widgets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
# contains helper functions to do actual optimizaiton analysis
import optimization_helper_functions as OHF

In [4]:
cwd = os.getcwd()

In [5]:
diameter = 1.27 #cm
area = np.pi*(diameter/2)**2
Fconst=96485

molar_mass_LiV3O8 = 287.7607
density_LiV3O8 = 3.15              
density_carbon = 2.266
max_mAhg = 362

ed_col = 'Electrode Volumetric Energy Density (Wh/cm3)'
pd_col = 'Electrode Volumetric Power Density (W/cm3)'
aed_col = 'Electrode Areal Energy Density (Wh/cm2)'
apd_col = 'Electrode Areal Power Density (W/cm2)'
ced_col = 'Cell Volumetric Energy Density (Wh/cm3)'
cpd_col = 'Cell Volumetric Power Density (W/cm3)'

In [6]:
# def add_anode(sims,L_sep=20,L_cc_a=15,L_cc_c=15,Q_spec_a=372,rho_a=2.2,eps_a=0.35,Q_a_Q_c_ratio=1):
def add_anode(sims,L_sep=20,L_cc_a=15,L_cc_c=15,Q_spec_a=3860,rho_a=0.534,eps_a=0.0,Q_a_Q_c_ratio=1):

    LBOC = L_sep+L_cc_a+L_cc_c
    
    sims['L_anode (um)'] = 10000*max_mAhg*sims['AM Loading (g)']/(Q_a_Q_c_ratio*Q_spec_a*area*(1-eps_a)*rho_a)
    sims['L_cell (um)'] = sims['L_anode (um)'] + LBOC + sims['L (um)']
    
    sims[ced_col] = sims[aed_col]/((sims['L (um)']+LBOC+sims['L_anode (um)'])/10000)
    sims[cpd_col] = sims[apd_col]/((sims['L (um)']+LBOC+sims['L_anode (um)'])/10000)
    
    return sims

In [7]:
drop_cols = ['Bind Loading (g)', 'Cathode Volume (cm3)','Vol Frac Bind','mAh_exp','tstep (s)',
             'Current (A)','Tortuosity (Bruggeman)']
crates = [0.1,0.2,0.33,0.5,0.75,1]
all_crate_sims_that_ran = []
for cr in crates:

    colname_filename = cwd+'/vary_crate/'+str(cr)+'_C/column_names.txt'

    filename = cwd+'/vary_crate/'+str(cr)+'_C/Simulation_Parameters_processed.txt'
    all_sims, sims_that_ran, sims_that_crashed = OHF.read_in_sims_table(filename,colname_filename,drop_cols,print_cols=False)

    sims_that_ran[apd_col] = sims_that_ran[pd_col]*sims_that_ran['L (um)']/10000
    all_crate_sims_that_ran.append(sims_that_ran)

all_sims_that_ran = pd.concat(all_crate_sims_that_ran)
all_sims_that_ran = all_sims_that_ran.drop(['fail_id','Time','Voltage','Integrated Voltage','Ran','Utilization_cr'],axis=1)
all_sims_that_ran = add_anode(all_sims_that_ran)

The details of this analysis can be found in the paper [Design Principles to Govern Electrode Fabrication for the Lithium Trivanadate Cathode](https://iopscience.iop.org/article/10.1149/1945-7111/ab91c8). 

# Optimal Design

Choose the **optimization metric** (i.e. Cell Energy Density) you wish to optimize. 
The **sensitivity** will be displayed as a shaded region that will show the range of design parameter values for which the designated fraction of the maximum achievable **optimization metric** is achieved when the other design parameters are held at their optimal values.
After the desired **optimization metric** and **sensitivity** are chosen, click the **Run Interact** button to update the plot. 

In [23]:
# plt.close(figi)
figi,axi = plt.subplots(1,3,figsize=(9,2.5),constrained_layout=True)
@widgets.interact_manual(optimization_metric=[ed_col,aed_col,ced_col,pd_col,apd_col,cpd_col],sensitivity=(0.5,1,0.05))
def plot_optimization(optimization_metric,sensitivity):
    for ax in axi:
        ax.clear()
    
    # load all data
    drop_cols = ['Bind Loading (g)', 'Cathode Volume (cm3)','Vol Frac Bind','mAh_exp','tstep (s)',
             'Current (A)','Tortuosity (Bruggeman)']
    crates = [0.1,0.2,0.33,0.5,0.75,1]
    all_crate_sims_that_ran = []
    for cr in crates:

        colname_filename = cwd+'/vary_crate/'+str(cr)+'_C/column_names.txt'

        filename = cwd+'/vary_crate/'+str(cr)+'_C/Simulation_Parameters_processed.txt'
        all_sims, sims_that_ran, sims_that_crashed = OHF.read_in_sims_table(filename,colname_filename,drop_cols,print_cols=False)

        sims_that_ran[apd_col] = sims_that_ran[pd_col]*sims_that_ran['L (um)']/10000
        all_crate_sims_that_ran.append(sims_that_ran)

    all_sims_that_ran = pd.concat(all_crate_sims_that_ran)
    all_sims_that_ran = all_sims_that_ran.drop(['fail_id','Time','Voltage','Integrated Voltage','Ran','Utilization_cr'],axis=1)
    all_sims_that_ran = add_anode(all_sims_that_ran)
        
        
    design_parameter = optimization_metric
    best_cols = ['Porosity','Vol Frac Cond','L (um)']
    output_labels = [r'Cell $E_V$ ($\frac{Wh}{L}$)',r'$E_A$ ($\frac{Wh}{m^2}$)']

    all_sims_that_ran,new_design_parameter = OHF.change_units(all_sims_that_ran,design_parameter)
    best_df = OHF.design_guide_plots(all_sims_that_ran,best_cols,[axi[0],axi[1],axi[2]],optimize_output=new_design_parameter,color='blue')
    # for pp in [0.9]:
    pp = sensitivity
    OHF.plot_percentile(all_sims_that_ran,best_cols,[axi[0],axi[1],axi[2]],color='blue',percentile=pp,optimize_output=new_design_parameter)


    axi[0].set_ylim(0,1)
    axi[0].set_ylabel(r'Optimal $\epsilon$')

    axi[1].axhline(y=0.05,linestyle='--',color='black',lw=2,marker='')
    axi[1].set_yscale('linear')
    axi[1].set_ylim(0.01,0.09)
    axi[1].set_ylabel(r'Optimal $\widebar{v}_{Cond}$')

    axi[2].set_ylim(0,np.min([1.25*axi[2].get_ylim()[1],500]))
    axi[2].set_ylabel(r'Optimal L ($\mu m$)')


    for axicr in figi.axes:
        axicr.set_xlim(0.1,1.0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='optimization_metric', options=('Electrode Volumetric Energy Densit…

In [9]:
class electrode:
    chemistry = 'LVO'
    
    # Initializer / Instance Attributes
    def __init__(self, L, eps, vfc, vfb=0.0):
        self.L = L
        self.eps = eps
        self.vfc = vfc
        self.vfb = vfb
        
    def get_specs(self):
        out_dict = {}
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            out_dict[col] = val
        return out_dict
        
    def nearest_simulated_electrode(self):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        return sims
        
    def performance(self,c_rate):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        return sims.loc[sims['C-rate (1/h)']==c_rate]
        
    def rate_capability(self,ax=None):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        if ax is not None:
            lab = 'L = '+str(int(sims['L (um)'].values[0]))+r', $\epsilon$ =' + str(np.round(sims['Porosity'].values[0],2))+r', $\widebar{v}_{CNT}$ ='+ str(np.round(sims['Vol Frac Cond'].values[0],2))
            ax.plot(sims['C-rate (1/h)'],sims['Capacity (mAh/g)'],'-o',label=lab)
            ax.set_xlabel('C-rate (1/h)')
            ax.set_ylabel('Capacity (mAh/g)')
        return sims[['C-rate (1/h)','Capacity (mAh/g)']]
#     def active_material_mass(self):
    
#     def conductor_mass(self):
        
#     def binder_mass(self):
        
#     def active_material_mass_loading(self):
        
#     def volume_distribution(self):
        
#     def tortuosity(self):
        
#     def electronic_conductivity(self):
        
# class cell(electrode):
    

# Rate Capability

This module allows the user to virtually "make" different electrodes and visualize how the discharge capacity will depend on the design parameters chosen. 

The length of the electrode in micros **L_um**, the electrode **porosity**, and the volume fraction of conductive additive **vol_frac_cond** must be specified. 

After the desired values of design parameters are chosen, click the **Run Interact** button to update the plot. 

The plot can be cleared by checking the **clear_plot** box.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4),constrained_layout=True)
fig.suptitle('Rate Capability')
@widgets.interact_manual(porosity=(0.1, 0.9,0.01),L_um=(50,500,10),vol_frac_cond=(0.01,0.2,0.01),Clear_Plot=False)
# @widgets.interact_manual(choose_eps,choose_L,choose_vfc)
def plot_RC(L_um,porosity,vol_frac_cond,Clear_Plot):
    if Clear_Plot:
        ax.clear()
    e1 = electrode(L=L_um,eps=porosity,vfc=vol_frac_cond)
    RC_df = e1.rate_capability(ax)
    ax.grid(True)
    
    colors = plt.cm.plasma(np.linspace(0.3,1,len(ax.lines)))
    for l,c in zip(ax.lines,colors):
        l.set_color(c)
        
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=270, description='L_um', max=500, min=50, step=10), FloatSlider(value=0.…